# BC Data Collection

In this notebook, we'll programmatically get data from Business Central through two API endpoints, the `ODataV4` and `API v2` endpoints.

1. **ODataV4 Endpoint**

    This endpoint is of the form:
    
    `https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/ODataV4/`

    It is a standards-based OData 4.0 protocol endpoint for data access, especially for reading, filtering, and updating data entities in BC. Generally,  it's used to query and manipulate Business Central data like tables and pages that are published as web services.

    > Note: Use ODataV4 when you need direct access to Business Central pages/tables that you’ve published as web services.

2. **API v2 Endpoint**

    This endpoint has the form:

    `https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/api/v2.0/`

    It is a standardized API layer built by Microsoft for Business Central that uses OData v4 under the hood, but with stable, versioned, REST-style endpoints. This endpoint is designed for programmatic integrations, connectors, and third-party applications (like Power Automate, Power BI, or custom apps).

    > Note: Use API v2.0 (or later) when building integrations or apps — it’s versioned, stable, and standardized.


## The BC API Utility

In general, getting data from BC is a two-step process. You first need to get a Bearer token that will be used for authentication. Only select accounts can get this and you will need to enter a password as well. Next, you will have to pass this token to another request. This time you need to select the URL (or API endpoint) which will handle your specific data requirement and provide the token as part of the request headers.

To do this, we built the BC API utility to make the process a bit more streamlined, especially for integration in our Airflow scripts.

This contains two classes:

1. `BCTokenClient` which helps us get authentication tokens for further access to BC data. 
2. `BCHandler` which allows us to specify the data we're requesting. We'll pass the token when we first build the class so it would authenticate all of our subsequent requests. The token expires after sometime, to which you'll need to create another one using the `BCTokenClient`.

The `BCHandler` class can now handle both `ODataV4` and `API v2` endpoints.

In [1]:
import os
from bc_api.authenticator import BCTokenClient 
from bc_api.handler import BCHandler
from dotenv import load_dotenv

import pandas as pd
pd.set_option('display.max_columns', None)

We stored the variables in the `.env` file. This way, we avoid hard coding any secrets inside the notebook. Another good thing is that if we use git, we can set this file inside the `.gitignore` config file, protecting us from any scenario where we might accidentally share secret information, such as passwords and keys.

In [2]:
# Load credentials from .env file
def load_credentials():
    """Load credentials from environment variables."""
    username = os.getenv('BC_USERNAME') or os.getenv('USER')
    password = os.getenv('BC_PASSWORD') or os.getenv('PASSWORD')
    
    if not username:
        raise ValueError("Username not found. Please set BC_USERNAME or USER environment variable.")
    
    if not password:
        raise ValueError("Password not found. Please set BC_PASSWORD or PASSWORD environment variable.")
    
    return username, password


We begin by instantiating the `BCTokenClient` and `BCHandler` classes.

In [ ]:
username, password = load_credentials()

token = BCTokenClient().get_token(username=username, password=password)

# Inspect the token
print(f'Authentication Token: {token}')

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.70.231'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Authentication Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6InlFVXdtWFdMMTA3Q2MtN1FaMldTYmVPYjNzUSIsImtpZCI6InlFVXdtWFdMMTA3Q2MtN1FaMldTYmVPYjNzUSJ9.eyJhdWQiOiJodHRwczovL2FwaS5idXNpbmVzc2NlbnRyYWwuZHluYW1pY3MuY29tIiwiaXNzIjoiaHR0cHM6Ly9zdHMud2luZG93cy5uZXQvNDA3NzNiZDgtZDlmZS00NzQ1LTlkNDQtODUxZjgxOTEyZWI0LyIsImlhdCI6MTc2MjgzMDUzNCwibmJmIjoxNzYyODMwNTM0LCJleHAiOjE3NjI4MzQ0MzQsImFpbyI6ImsySmdZTWg0SmRoNVpzM01vOW8vbHYyeDdEdVRDQUE9IiwiYXBwaWQiOiIyYTBhZjhjZC1iZGY0LTQ0MTUtOTc3OC00Mjk0ODRjZTU0MDgiLCJhcHBpZGFjciI6IjEiLCJpZHAiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC80MDc3M2JkOC1kOWZlLTQ3NDUtOWQ0NC04NTFmODE5MTJlYjQvIiwiaWR0eXAiOiJhcHAiLCJvaWQiOiI3ZWJiMDBlZi03YzEwLTRmMzAtYjg5OS00M2RiMTJkZGZlODQiLCJyaCI6IjEuQVhJQTJEdDNRUDdaUlVlZFJJVWZnWkV1dEQzdmJabHNzMU5CaGdlbV9Ud0J1Sjh3QVFCeUFBLiIsInJvbGVzIjpbImFwcF9hY2Nlc3MiLCJBUEkuUmVhZFdyaXRlLkFsbCJdLCJzdWIiOiI3ZWJiMDBlZi03YzEwLTRmMzAtYjg5OS00M2RiMTJkZGZlODQiLCJ0aWQiOiI0MDc3M2JkOC1kOWZlLTQ3NDUtOWQ0NC04NTFmODE5MTJlYjQiLCJ1dGkiOiJwU0JpM0ZwSDkwYUJqemVLWWVpaEFBIiwid

Now, let's pass the token to the handler. Remember, it's the `BCHandler` class that will allow us to retrieve data from BC.

In [33]:
tenant_id = '40773bd8-d9fe-4745-9d44-851f81912eb4'
env_name = 'DaconProduction'
endpoint_type = 'ODataV4'

# Pass all the parameters
bc = BCHandler(tenant_id=tenant_id, env_name=env_name, endpoint_type=endpoint_type, token=token)

Let's request data!

Let's start by requesting metadata and companies. It's good to start with these since these allow us to determine:

1. the actual names of the tables that we can query in the environment, and 
2. the correct name of the companies. 

Since these are quite special, we placed them inside a special class called `BCMetaData` and `BCData`. These classes take in the raw data and gives us built-in data processing capabilities. Once we have these, we can use the following attributes:

| Attribute | Description | Usage |
| --------- | ----------- | ----- |
| `raw` | The main response, whether in XML (for metadata) or JSON (for all other responses).  | `metadata.raw` |
| `json` | Only available for `BCMetaData`. Returns the JSON conversion of the XML metadata file | `metadata.json` |
| `flat_json` | The flattened JSON file which can be further processed into a table format in pandas through `pd.json_normalize` | `metadata.flat_json` |

In [34]:
metadata = bc.get_metadata()
pd.json_normalize(metadata.flat_json)

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.businesscentral.dynamics.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,API Endpoint Name,Column Name,DataType,Nullable,MaxLength,Scale,Annotations.NAV.LabelId,Annotations.NAV.NavType,Annotations.NAV.AllowEditOnCreate,Annotations.NAV.AllowEdit
0,Media,ID,Edm.Guid,false,None,None,ID,NAV.NavType/Guid,true,NaN
1,Media,timestamp,Edm.Int64,None,None,None,timestamp,NAV.NavType/Long,false,false
2,Media,Description,Edm.String,None,250,None,Description,NAV.NavType/String,NaN,NaN
3,Media,Content,Edm.Stream,None,None,None,Content,NAV.NavType/Binary,NaN,NaN
4,Media,Mime_Type,Edm.String,None,100,None,Mime Type,NAV.NavType/String,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4569,VendorList,no,Edm.String,false,20,None,No.,NAV.NavType/String,NaN,false
4570,VendorList,name,Edm.String,None,100,None,Name,NAV.NavType/String,false,false
4571,VendorList,name2,Edm.String,None,50,None,Name 2,NAV.NavType/String,false,false
4572,VendorList,address,Edm.String,None,100,None,Address,NAV.NavType/String,false,false


In [35]:
companies = bc.get_companies()
pd.json_normalize(companies.flat_json)

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.businesscentral.dynamics.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Name,timestamp,Evaluation_Company,Display_Name,Id,Business_Profile_Id,SystemCreatedAt,SystemCreatedBy,SystemModifiedAt,SystemModifiedBy
0,Dacon Corporation,1899449,False,Dacon Corporation,887e96e3-1cbc-ef11-b8ea-000d3a091610,,2024-12-17T02:16:21.1Z,e96607c5-f1d2-4a90-9287-c3724517d6c7,2024-12-17T02:16:58.943Z,e96607c5-f1d2-4a90-9287-c3724517d6c7


Now, we can see the full list of tables that are available in the endpoint that we specified, together with the API endpoint name. Now, we can specify the company name (from the Company table) and the table name (from the Metadata Table) in the vanilla `get_data` function.

> Note: The `get_data` function only works for the `ODataV4` endpoint as of now.

In [37]:
company_name = 'Dacon Corporation'
table_name = 'VendorLedgerEntries'

data = bc.get_data(company_name, table_name)

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.businesscentral.dynamics.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Now, let's parse the results from the BC dataframe.

In [38]:
pd.json_normalize(data.get('value', []))

,Entry_No,Transaction_No,Vendor_No,Posting_Date,Due_Date,Pmt_Discount_Date,Document_Date,Document_Type,Document_No,Purchaser_Code,Source_Code,Reason_Code,IC_Partner_Code,Open,Currency_Code,Dimension_Set_ID,Amount,Debit_Amount,Credit_Amount,Remaining_Amount,Amount_LCY,Debit_Amount_LCY,Credit_Amount_LCY,Remaining_Amt_LCY,Original_Amt_LCY,Vendor_Name,AuxiliaryIndex1
0,6,3,SH00001,2025-01-20,2025-01-20,0001-01-01,2025-01-20,,AR00002671,,CASHRECJNL,,,True,,30,-8333.33,0.00,8333.33,-8333.33,-8333.33,0.00,8333.33,-8333.33,-8333.33,SHAREHOLDER,SH00001
1,18,4,V00057,2025-01-06,2025-01-06,2025-01-06,2025-01-06,Invoice,PPIN00000001,,PURCHASES,,,False,,42,-26296.34,0.00,26296.34,0.00,-26296.34,0.00,26296.34,0.00,-26296.34,CANON MARKETING (PHILIPPINES) INC.,V00057
2,22,6,V00378,2025-01-09,2025-01-09,0001-01-01,2025-01-09,Payment,1862364,,PAYMENTJNL,,,False,,80,100800.00,100800.00,0.00,0.00,100800.00,100800.00,0.00,0.00,100800.00,VASQUEZ BRETAÑA & SIBAL-PULIDO LAW OFFICES,V00378
3,26,8,V00360,2025-01-15,2025-01-15,0001-01-01,2025-01-15,Payment,1862368,,PAYMENTJNL,,,False,,105,52063.45,52063.45,0.00,0.00,52063.45,52063.45,0.00,0.00,52063.45,THEOS HELIOS SECURITY AGENCY CORP,V00360
4,34,12,V00057,2025-01-16,2025-01-16,0001-01-01,2025-01-16,Payment,1862373,,PAYMENTJNL,,,False,,168,26296.34,26296.34,0.00,0.00,26296.34,26296.34,0.00,0.00,26296.34,CANON MARKETING (PHILIPPINES) INC.,V00057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2662,12667,3709,V00010,2025-10-01,2025-10-01,0001-01-01,2025-10-01,,JV00000750,,PAYMENTJNL,,,False,,5505,278301.59,278301.59,0.00,0.00,278301.59,278301.59,0.00,0.00,278301.59,AMALGAMATED INVESTMENT BANCORPORATION,V00010
2663,12668,3709,V00010,2025-10-01,2025-10-01,0001-01-01,2025-10-01,,JV00000750,,PAYMENTJNL,,,False,,5938,-146250.00,0.00,146250.00,0.00,-146250.00,0.00,146250.00,0.00,-146250.00,AMALGAMATED INVESTMENT BANCORPORATION,V00010
2664,12669,3710,V00010,2025-10-10,2025-10-10,0001-01-01,2025-10-10,,JV00000718,,REVERSAL,,,False,,5938,156000.00,0.00,-156000.00,0.00,156000.00,0.00,-156000.00,0.00,156000.00,AMALGAMATED INVESTMENT BANCORPORATION,V00010
2665,12670,3710,V00010,2025-10-10,2025-10-10,0001-01-01,2025-10-10,,JV00000718,,REVERSAL,,,False,,5505,-227544.15,-227544.15,0.00,0.00,-227544.15,-227544.15,0.00,0.00,-227544.15,AMALGAMATED INVESTMENT BANCORPORATION,V00010


The guide ends here.

# Scratch Paper

In [32]:
import requests
import xml.etree.ElementTree as ET

In [ ]:
# Construction Patterns
tenant_id = '40773bd8-d9fe-4745-9d44-851f81912eb4'
# env_name = 'DaconProduction'
env_name = 'SEMIRARA_TEST'
token = BCTokenClient().get_token(username=username, password=password)

# ODataV4 endpoint construction patterns
# endpoint = f'https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/ODataV4/$metadata'      ## Metadata
# endpoint = f"https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/ODataV4/Company"        ## Company
# endpoint = f"https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/ODataV4/Company('BASE')/Vendor_Card_Excel"      ## Table name and company name specified

# API v2 endpoint construction patterns
# endpoint = f"https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/api/v2.0/$metadata"
# endpoint = f"https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/api/v2.0/companies"
# endpoint = f"https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/api/v2.0/companies(887e96e3-1cbc-ef11-b8ea-000d3a091610)/purchaseOrders"
# endpoint = f"https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/api/v2.0/companies(887e96e3-1cbc-ef11-b8ea-000d3a091610)/purchaseOrders(d7e8ba5b-fa18-f011-9af4-0022485613ed)/purchaseOrderLines"

# endpoint = f"https://api.businesscentral.dynamics.com/v2.0/{tenant_id}/{env_name}/api/v2.0/companies(df434a54-a3ee-ef11-9345-6045bd20ee6d)/purchaseOrderLines"        ## will not work, purchase order must be specified

headers = {'Authorization': f'Bearer {token}'}

# data = requests.get(endpoint, headers=headers).content.decode('utf-8')    # for metadata only
# data = requests.get(endpoint, headers=headers).json()                     # for all other datasets

c:\Users\laguidote\Documents\2025 Projects\bc-conn\.venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host '192.168.70.231'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'@odata.context': 'https://api.businesscentral.dynamics.com/v2.0/40773bd8-d9fe-4745-9d44-851f81912eb4/SEMIRARA_TEST/ODataV4/$metadata#Company',
 'value': [{'Name': 'BASE',
   'timestamp': 5168521,
   'Evaluation_Company': False,
   'Display_Name': 'BASE-SEMIRARA',
   'Id': 'df434a54-a3ee-ef11-9345-6045bd20ee6d',
   'Business_Profile_Id': '',
   'SystemCreatedAt': '2025-02-19T09:24:34.317Z',
   'SystemCreatedBy': 'e96607c5-f1d2-4a90-9287-c3724517d6c7',
   'SystemModifiedAt': '2025-02-23T10:53:49.417Z',
   'SystemModifiedBy': 'e96607c5-f1d2-4a90-9287-c3724517d6c7'},
  {'Name': 'CERI- Manila',
   'timestamp': 39378748,
   'Evaluation_Company': False,
   'Display_Name': 'CERI LOGISTICS, INC.',
   'Id': '0a153d7f-9aad-f011-bbd0-000d3ac6e11f',
   'Business_Profile_Id': '',
   'SystemCreatedAt': '2025-10-20T09:52:33Z',
   'SystemCreatedBy': 'e96607c5-f1d2-4a90-9287-c3724517d6c7',
   'SystemModifiedAt': '2025-10-22T08:26:38.847Z',
   'SystemModifiedBy': 'e96607c5-f1d2-4a90-9287-c3724517d6c7'}